In [ ]:

# Tokenize input
input_texts = df_train['train.SRC'].head(10000)
output_texts = df_train['train.EXR'].head(10000)

input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_texts)
input_sequences = input_tokenizer.texts_to_sequences(input_texts)
input_vocab_size = len(input_tokenizer.word_index) + 1

# Pad input sequences
max_input_length = max(len(seq) for seq in input_sequences)
input_sequences_padded = pad_sequences(input_sequences, maxlen=max_input_length, padding='post')

output_tokenizer = Tokenizer(filters="")
output_tokenizer.fit_on_texts(output_texts)
output_sequences = output_tokenizer.texts_to_sequences(output_texts)
output_vocab_size = len(output_tokenizer.word_index) + 1

# Pad output sequences
max_output_length = max(len(seq) for seq in output_sequences)
output_sequences_padded = pad_sequences(output_sequences, maxlen=max_output_length, padding='post')

# Ensure both input and output sequences have the same length
max_length = max(max_input_length, max_output_length)
input_sequences_padded = pad_sequences(input_sequences, maxlen=max_length, padding='post')
output_sequences_padded = pad_sequences(output_sequences, maxlen=max_length, padding='post')

# Encoder
encoder_input = Input(shape=(max_length,))
encoder_embedding = Embedding(input_dim=input_vocab_size, output_dim=128)(encoder_input)
encoder_lstm, state_h, state_c = LSTM(256, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_input = Input(shape=(max_length-1,))
decoder_embedding = Embedding(input_dim=output_vocab_size, output_dim=128)(decoder_input)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_output, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(output_vocab_size, activation='softmax')
decoder_output = decoder_dense(decoder_output)

model = Model([encoder_input, decoder_input], decoder_output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Correctly shape the decoder input and output data
decoder_input_data = output_sequences_padded[:, :-1]  # Remove the last token
decoder_output_data = output_sequences_padded[:, 1:]  # Remove the first token


model.fit(
    [input_sequences_padded, decoder_input_data],
    decoder_output_data,
    batch_size=32,
    epochs=10
)


In [ ]:
# Encoder Model
encoder_model = Model(encoder_input, encoder_states)

# Decoder Model
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm_output, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_output = decoder_dense(decoder_lstm_output)

decoder_model = Model(
    [decoder_input] + decoder_states_inputs,
    [decoder_output] + decoder_states
)

def decode_sequence(input_seq):
    # Ensure input sequence is padded correctly
    input_seq = pad_sequences(input_seq, maxlen=max_length, padding='post')  # Padding to max_input_length
    
    states_value = encoder_model.predict(input_seq)

    # Start token
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = output_tokenizer.word_index["(order"]  # Use a start token

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        if sampled_token_index == 0:
            break
        sampled_token = output_tokenizer.index_word[sampled_token_index]
        decoded_sentence += " " + sampled_token

        # if sampled_token == ")" or len(decoded_sentence) > max_output_length:
        #     stop_condition = True

        # if len(decoded_sentence) > max_output_length+30:
        #     stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        states_value = [h, c]

    return decoded_sentence

test_input_sequence = input_tokenizer.texts_to_sequences(df_dev['dev.SRC'].head(1))
output = decode_sequence(test_input_sequence)
print(output)